### **BÀI THI SỐ 2: PHÂN LOẠI NHÃN EMAIL**

**Import các thư viện cần thiết**

In [84]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [85]:
import pandas as pd
from typing import Tuple, List, Dict
from tabulate import tabulate
from sklearn.preprocessing import LabelEncoder #Chuyển nhãn phân loại thành giá trị số
import matplotlib.pyplot as plt #Vẽ biểu đồ
import seaborn as sns #Vẽ biểu đồ
import numpy as np #Tính toán và mảng, ma trận
import string
import nltk
import pandas as pd
from nltk.corpus import stopwords #Từ dừng: hiểu đơn giản là các từ có tần số xuất hiện nhiều như the, to... các từ này thường mang ít giá trị ý nghĩa và không khác nhau nhiều trong các văn bản khác nhau.
from sklearn import metrics #Đánh giá hiệu suất mô hình
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split #Chia thành tập train, tập test
import os

**Đọc dữ liệu và tạo DataFrame**

In [86]:
notspam = np.arange(0, 193) #[0, 1, 2, ... ,193]
spam = np.arange(0, 18)

df_list = [] #Lưu các DataFrame đọc được

for ns in notspam :
    df_tmp1 = pd.read_csv('/content/drive/My Drive/BaiThi2_Samsung/TrainData/notspam/{}_notspam.txt'.format(ns))
    df_tmp1.drop(df_tmp1.columns[[0]], axis = 1, inplace = True)
    f = open('/content/drive/My Drive/BaiThi2_Samsung/TrainData/notspam/{}_notspam.txt'.format(ns), "r")
    df_tmp1['message'] = f.read()
    df_tmp1['Label'] = 'notspam'
    df_list.append(df_tmp1)

for s in spam :
    df_tmp2 = pd.read_csv('/content/drive/My Drive/BaiThi2_Samsung/TrainData/spam/{}_spam.txt'.format(s))
    df_tmp2.drop(df_tmp2.columns[[0]], axis = 1, inplace = True)
    f2 = open('/content/drive/My Drive/BaiThi2_Samsung/TrainData/spam/{}_spam.txt'.format(s), "r")
    df_tmp2['message'] = f2.read()
    df_tmp2['Label'] = 'spam'
    df_list.append(df_tmp2)

In [87]:
df = pd.concat(df_list).reset_index(drop = True) #concat: nối các dataframe trong list thành 1 DataFrame duy nhất, reset_index: Thiết lập chỉ số mới, drop: loại bỏ chỉ số cũ
df = df.iloc[ : , 0 : 2] #:, giữ tất cả các hàng, 0:2: Lấy từ [0;2)
df

,message,Label
0,Subject: re : 2 . 882 s - > np np\n\n> date : ...,notspam
1,Subject: s - > np + np\n\ndiscussion s - > np ...,notspam
2,Subject: 2 . 882 s - > np np\n\n. . . 's much ...,notspam
3,"Subject: gent conference\n\n"" listserv "" inter...",notspam
4,Subject: query : causatives korean\n\nanyone p...,notspam
...,...,...
206,Subject: `\n\nhello ! our naughty little city ...,spam
207,Subject: attract women ! ! ( 30740 )\n\nattrac...,spam
208,Subject: are serious . . . help retire 2 3 yea...,spam
209,"Subject: "" complete home business kit ! ! ""\n\...",spam


**Xử lý chuỗi văn bản thô**

In [88]:
def text_preprocess(mess):
    #Loại bỏ dấu câu
    nopunc = []
    for char in mess:
        if char not in string.punctuation: nopunc.append(char)
    #Chuyển list thành string
    nopunc = ''.join(nopunc)
    nopunc = nopunc.lower()
    #Tách từ & Loại bỏ các từ dừng
    words = nopunc.split()
    nostop = []
    for word in words:
        if word not in stopwords.words('english'): nostop.append(word)
    return nostop

**Lọc tin nhắn spam và notspam (Build DataFrame)**

In [6]:
spam_messages = df[df["Label"] == "spam"]["message"]
notspam_messages = df[df["Label"] == "notspam"]["message"]
display(spam_messages)
display(notspam_messages)

,message
193,Subject: great part-time summer job !\n\n* * *...
194,Subject: auto insurance rates too high ?\n\nde...
195,Subject: advertsing ? legal ! ! offer smtp ! !...
196,Subject: free trial membership\n\nlatest adult...
197,"Subject: market millions , try free\n\nmessage..."
198,Subject: lists software worldwide\n\norder for...
199,Subject: zero down internet opportunity !\n\n$...
200,"Subject: re : free !\n\nhello , are offering f..."
201,"Subject: \n\ncomes porn , site does n't mess a..."
202,Subject: even steal identity !\n\nare being in...


,message
0,Subject: re : 2 . 882 s - > np np\n\n> date : ...
1,Subject: s - > np + np\n\ndiscussion s - > np ...
2,Subject: 2 . 882 s - > np np\n\n. . . 's much ...
3,"Subject: gent conference\n\n"" listserv "" inter..."
4,Subject: query : causatives korean\n\nanyone p...
...,...
188,Subject: english snow words\n\njonathan david ...
189,Subject: news ippe ( 04 nov 94 )\n\nn e w s f ...
190,Subject: translator\n\norder forms translator ...
191,"Subject: special issues\n\nnames , journal ame..."


**Lưu lại tất cả các từ spam (Trả về list các từ spam)**

In [7]:
#Tải bộ từ dừng
import nltk
nltk.download('stopwords')

# Words in spam messages
spam_words = []
for each_message in spam_messages:
    spam_words += text_preprocess(each_message)
print(spam_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['subject', 'great', 'parttime', 'summer', 'job', 'display', 'boxes', 'credit', 'applications', 'need', 'place', 'small', 'owneroperated', 'stores', 'area', '1', 'introduce', 'store', 'owner', 'manager', '2', '90', 'effective', 'script', 'tells', 'little', 'display', 'box', 'save', 'customers', 'hundreds', 'dollars', 'drawing', 'card', 'business', '5', '00', '15', '00', 'every', 'app', 'sent', '3', 'spot', 'counter', 'place', 'box', 'nothing', 'need', 'done', 'need', 'name', 'address', 'company', 'send', 'commission', 'checks', 'compensaation', '10', 'every', 'box', 'place', 'becoming', 'representative', 'earn', 'commission', '10', 'application', 'came', 'store', 'course', 'much', 'profitable', 'plan', 'pay', 'months', 'years', 'small', 'effort', 'call', '1888', '7035390', 'code', '3', '24', 'hours', 'receive', 'details', 'removed', 'mailing', 'list', 'type', 'b2998', 'hotmail', 'com', 'area', 'remove', 'subject', 'area', 'e', 'mail', 'send', 'subject', 'auto', 'insurance', 'rates', 'h

**Xử lý văn bản và chuyển thành feature vector**

In [50]:
df1 = df.copy()
df1.drop(columns=["Label"], inplace = True)
df1

,message
0,subject 2 882 np np date sun 15 dec 91 02 25 0...
1,subject np np discussion np np reminds years a...
2,subject 2 882 np np much restrictive np np np ...
3,subject gent conference listserv international...
4,subject query causatives korean anyone point b...
...,...
206,subject hello naughty little city made especia...
207,subject attract women 30740 attract women natu...
208,subject serious help retire 2 3 years money wa...
209,subject complete home business kit earn 70 000...


In [89]:
df1["message"] = df1["message"].apply(text_preprocess)
#Chuyển list thành chuỗi có dấu cách
def concatenate_words(words):
    return " ".join(words)
df1["message"] = df1["message"].apply(concatenate_words)

vectorizer = CountVectorizer() #Chuyển văn bản thành ma trận tần suất từ vựng
bow_transformer = vectorizer.fit(df1["message"])  # Huấn luyện và xây dựng từ điển
messages_bow = bow_transformer.transform(df1["message"]) # Chuyển đổi các văn bản thành ma trận tần suất từ vựng
messages_bow_array = messages_bow.toarray()
print(messages_bow_array)

[[ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [21 14  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]


In [69]:
df1.shape

(211, 1)

**Tính TF - IDF từ ma trận tần số**

In [70]:
# TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer().fit(messages_bow)

#Chuyển đổi BoW thành ma trận TF-IDF
messages_tfidf = tfidf_transformer.transform(messages_bow)
print(messages_tfidf.shape)

(211, 11248)


In [71]:
FactorResult = pd.factorize(df["Label"])
df["Label"] = FactorResult[0]
df
# Ví dụ:
# 'Email ID': [1, 2, 3, 4, 5],
# 'Label': ['spam', 'not spam', 'spam', 'not spam', 'spam']
# --->FactorResult = (array([0, 1, 0, 1, 0]), Index(['spam', 'not spam'], dtype='object'))


,message,Label
0,subject 2 882 np np date sun 15 dec 91 02 25 0...,0
1,subject np np discussion np np reminds years a...,0
2,subject 2 882 np np much restrictive np np np ...,0
3,subject gent conference listserv international...,0
4,subject query causatives korean anyone point b...,0
...,...,...
206,subject hello naughty little city made especia...,1
207,subject attract women 30740 attract women natu...,1
208,subject serious help retire 2 3 years money wa...,1
209,subject complete home business kit earn 70 000...,1


**Chia tập dữ liệu thành tập huấn luyện và tập kiểm tra**

In [56]:
msg_train, msg_test, label_train, label_test = train_test_split(
    messages_tfidf, df["Label"], test_size=0.2
)

**Xử lý dữ liệu chưa có nhãn**

In [73]:
no_labels = np.arange(0, 78)
df_noLabel = []

for nl in no_labels:
    df_unknown = pd.read_csv('/content/drive/My Drive/BaiThi2_Samsung/TestData_nolabel/{}_unknown.txt'.format(nl))
    df_unknown.drop(df_unknown.columns[[0]], axis=1, inplace=True)
    with open('/content/drive/My Drive/BaiThi2_Samsung/TestData_nolabel/{}_unknown.txt'.format(nl), "r") as f:
        df_unknown['message'] = f.read()
    df_noLabel.append(df_unknown)

df_noLabel1 = pd.concat(df_noLabel).reset_index(drop=True)
df_noLabel1 = df_noLabel1.iloc[:, 0:1]
df_noLabel1

,message
0,Subject: base generated adjuncts\n\ndoes anyon...
1,Subject: basic journals\n\nare facing major cu...
2,Subject: query : tagalog philippine informants...
3,Subject: ancient vocal tract simulation\n\nwor...
4,"Subject: re : 3 . 386 chomsky , mac concordanc..."
...,...
73,Subject: really cool hot videos !\n\nattention...
74,"Subject: improvements wound care !\n\nhello , ..."
75,Subject: \n\nb r g n r f r e s 1 - stop travel...
76,"Subject: credit program "" guaranteed credit ""\..."


**Xử lý văn bản và chuyển thành vector feature**

In [74]:
df_noLabel1["message"] = df_noLabel1["message"].apply(text_preprocess)

# Chuyển list thành chuỗi có dấu cách
df_noLabel1["message"] = df_noLabel1["message"].apply(concatenate_words)

# Sử dụng vectorizer đã huấn luyện để biến đổi dữ liệu chưa có nhãn
messages_bow_NL = bow_transformer.transform(df_noLabel1["message"])
messages_tfidf_NL = tfidf_transformer.transform(messages_bow_NL)

messages_tfidf_NL.shape

(78, 11248)

**Sử dụng XGBoost để huấn luyện mô hình phân loại**

In [81]:
from xgboost import XGBClassifier
clf = XGBClassifier()
clf.fit(msg_train, label_train)

# Dự đoán
predict_train = clf.predict(msg_test)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

**Dự đoán dữ liệu chưa có nhãn và lưu kết quả**

In [91]:
model_pred = clf.predict(messages_tfidf_NL)
model_pred = pd.DataFrame(model_pred)
model_pred = model_pred.replace({0: 'nospam', 1: 'spam'})
model_pred.to_csv('/content/drive/My Drive/BaiThi2_Samsung/ANSWER.csv', index=False)